In [ ]:
# Initial development

# Import classes and methods
from pybrors.dicom import DicomFile, DicomDir

# Create file instance
tmp = DicomFile(file_path="../data/dicom/dicom_noimg_file.dcm")
print(tmp)
print(tmp.dataset["Modality"].value)

# Anonymize DICOM file
if tmp.anonymize():
    print("DICOM has been anonymized.")
else:
    print("DICOM has not been anonymized.")

print(tmp)
print(tmp.dataset["PatientName"])